In [ ]:
##Functional annotation based on KEGG

#POG fun s2g mog s2h

#assign fuctions to POGs and MOGs

cd /gpfs/project/raval/2023/MolAtlas_Reanalysed_June2023/Archaeplastida_clusters/Genomes/OrthoFinder/Results_Jun16/WorkingDirectory/OrthoFinder/Results_Jun23/MolAtlas/Functions/
import glob
import pandas as pd
import glob
import numpy as np
import os
import json
from collections import Counter

with open ('plants_geneID_to_KOID_dic.txt') as file:
    data = file.read()

geneID_to_KOID=json.loads(data)
geneID_to_KOID = {key.replace(':', '_'): value for key, value in geneID_to_KOID.items()}

with open('KOID_to_PWID_Dict.txt', 'r') as f:
    KOID_to_PWID = eval(f.read())

with open('PWId_to_PWname1.txt', 'r') as file:
    PWId_to_PWname1 = json.load(file)

with open('PWId_to_PWname2.txt', 'r') as f:
    PWId_to_PWname2 = eval(f.read())

with open('PWId_to_PWname3.txt', 'r') as f:
    PWId_to_PWname3 = eval(f.read())

with open('KOID_to_BRID.txt', 'r') as f:
    KOID_to_BRID = eval(f.read())

with open('BRID_to_BRname.txt', 'r') as f:
    BRID_to_BRname = eval(f.read())

with open('KOID_to_KOname_dic.txt', 'r') as f:
    KOID_to_KOname = eval(f.read())
##dictionary made based on files from KEGG
    


df=pd.read_csv('MOGs_protIDs.csv') ##GOG and POG will be loaded the same
df=df.set_index(['spcodes'])
df=df.drop(['Group'], axis=1)
df=df.T
df.to_csv('MOGs_protIDs_For_functions.csv')

infile='MOGs_protIDs_For_functions.csv'
OGID = []
KOID=[]
with open (infile) as f1:
    for ln in f1:
        OGID.append(ln.split(',')[0])
        ProtIDs=ln.split(',')[1:]
        ProtIDs=[item for item in ProtIDs if item.strip()]
        ProtIDs=[element.strip('" ') for element in ProtIDs]
        KOIDs=[]
        for i in ProtIDs:
            KOIDs.append(geneID_to_KOID.get(i))
        KOIDs=[x for x in KOIDs if x is not None]
        print(KOIDs)
        if not KOIDs:
            KOID.append('N/A')
        else:
            counter = Counter(KOIDs)
            KOID.append(counter.most_common(1)[0][0])
            print(counter.most_common(1)[0][0])
        
df = pd.DataFrame({'OGID': OGID, 'KOID': KOID})
df=df.iloc[1:,]

df['PathwayID'] = df['KOID'].map(KOID_to_PWID).fillna("N/A")
df['PWname1'] = df['PathwayID'].map(PWId_to_PWname1).fillna("N/A")
df['PWname2'] = df['PathwayID'].map(PWId_to_PWname2).fillna("N/A")
df['PWname3'] = df['PathwayID'].map(PWId_to_PWname3).fillna("N/A")

df['BRID'] = df['KOID'].map(KOID_to_BRID).fillna("N/A")
df['BRname'] = df['BRID'].map(BRID_to_BRname).fillna("N/A")
df['KOname'] = df['KOID'].map(KOID_to_KOname).fillna("N/A")

df.set_index('OGID', inplace=True)
df.loc['Functions not available'] = (df == "N/A").sum()
df.loc['Functions available'] = (df != "N/A").sum()
df.loc['Percentage Functions available'] = ((df != "N/A").sum() / df.shape[0]) * 100

df.to_csv('MOGs_Functions.csv') ##this file was manually inspected (for MOG and POG) and a plottable function was assigned based on functions present in each level


##repeat the same for POGs
##plottable categories are BRnames and PWname2 - from both N/As can be removed before plotting 
